# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [5]:
# Imports here
import torch
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [4]:
!wget "https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz"

--2023-02-27 14:47:37--  https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.131.8, 52.216.144.77, 54.231.199.184, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.131.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344873452 (329M) [application/x-gzip]
Saving to: 'flower_data.tar.gz'

flower_data.tar.gz  100%[===================>] 328.90M  73.5MB/s    in 4.4s    

2023-02-27 14:47:41 (74.3 MB/s) - 'flower_data.tar.gz' saved [344873452/344873452]



In [5]:
!tar -xf flower_data.tar.gz -C ./flowers

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [9]:
# TODO: Define your transforms for the training, validation, and testing sets
train_data_transforms = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_val_data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# TODO: Load the datasets with ImageFolder
train_datasets = ImageFolder(train_dir, transform=train_data_transforms)
test_datasets = ImageFolder(test_dir, transform=test_val_data_transforms)
val_datasets = ImageFolder(valid_dir, transform=test_val_data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloaders = DataLoader(train_datasets, batch_size=32, shuffle=True)
test_dataloaders = DataLoader(test_datasets, batch_size=32)
val_dataloaders = DataLoader(val_datasets, batch_size=32)

In [12]:
len(train_datasets.class_to_idx)

102

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [30]:
from torchvision.models import vgg19, alexnet, resnet34
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# TODO: Build and train your network

model = vgg19(num_classes=len(train_datasets.class_to_idx))


optimizer = optim.Adam(params=model.parameters(),lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

def train_model(dataloader, model, loss_fn, optimizer):
    losses = []
    correct_prediction = 0
    total_prediction = 0
    
    model.train()
    
    for item in tqdm(dataloader):
        inputs, labels = item
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        
        logits = torch.argmax(outputs, dim=1)
        correct_prediction += torch.sum(logits == labels).item()
        total_prediction += len(inputs)
        
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return (
        correct_prediction / total_prediction,
        torch.mean(torch.Tensor(losses)).item()
    )

def eval_model(dataloader, model, loss_fn):
    losses = []
    correct_prediction = 0
    total_prediction = 0
    
    model.eval()
    with torch.no_grad():
        for item in tqdm(dataloader):
            inputs, labels = item
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)

            logits = torch.argmax(outputs, dim=1)
            correct_prediction += torch.sum(logits == labels).item()
            total_prediction += len(inputs)

            losses.append(loss.item())
        
    return (
        correct_prediction / total_prediction,
        torch.mean(torch.Tensor(losses)).item()
    )
        
        


In [37]:
model.class_to_idx = train_datasets.class_to_idx

In [21]:
sample = next(iter(train_dataloaders))
out = model(sample[0])
out

tensor([[ 0.0398, -0.0082, -0.0525,  ..., -0.0816, -0.2154,  0.0656],
        [ 0.1703, -0.0285, -0.1458,  ..., -0.0487, -0.3144, -0.0856],
        [ 0.0727,  0.0338,  0.0883,  ..., -0.0335,  0.0003,  0.0788],
        ...,
        [ 0.1508,  0.0575, -0.0934,  ..., -0.1038, -0.1687,  0.0270],
        [ 0.1002,  0.0036, -0.0617,  ..., -0.1341, -0.2218,  0.1306],
        [ 0.2203,  0.0747,  0.0628,  ..., -0.0084, -0.1426,  0.0126]],
       grad_fn=<AddmmBackward0>)

In [22]:
torch.argmax(out,dim = 1)

tensor([ 37,  70,  61,  31,  37,  23,  61,  90, 101,   5,  87,  37,  43,  70,
         22,  61,  61,  79,  20,  31,  45,  70,  46,  43,  70,  45,  45,  32,
          0,  70,  79,  11])

In [31]:

def perform_training_loop(
    train_dataloader, 
    val_dataloader, 
    model, 
    loss_fn, 
    optimizer,
    EPOCH,
    train_acc_list = [],
    val_acc_list = [],
    train_loss_list = [],
    val_loss_list = [],
):

    for item in range(EPOCH):

        print(f'EPOCH : {item+1}/{EPOCH}')

        acc, loss = train_model(train_dataloaders, model, loss_fn, optimizer)
        train_acc_list.append(acc)
        train_loss_list.append(loss)

        print(f"Train loss = {loss}, train accuracy = {acc}")

        acc, loss = eval_model(val_dataloaders, model, loss_fn)
        val_acc_list.append(acc)
        val_loss_list.append(loss)

        print(f"Validation loss = {loss}, validation accuracy = {acc}")
        
    return {
        "train_acc": train_acc_list, 
        "val_acc": val_acc_list, 
        "train_loss": train_loss_list, 
        "val_loss": val_loss_list, 
    }
        
    
    

EPOCH : 1/2


  2%|▏         | 4/205 [02:35<2:10:09, 38.85s/it]


KeyboardInterrupt: 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [33]:
# TODO: Do validation on the test set

test_acc, test_loss = eval_model(test_dataloaders, model, loss_fn)



100%|██████████| 26/26 [03:20<00:00,  7.72s/it]


In [35]:
test_acc

0.018315018315018316

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [40]:
# TODO: Save the checkpoint 
model_name = f"vgg16_checkpoint"

def save_checkpoint(
    name,
    model, 
    optimizer,
    epoch,
    train_acc_list = [],
    val_acc_list = [],
    train_loss_list = [],
    val_loss_list = []
):

    checkpoint = {
        "model_state_dict":model.state_dict(),
        "optimizer_state_dict":optimizer.state_dict(),
        "train_acc_list":train_acc_list,
        "val_acc_list":val_acc_list,
        "train_loss_list":train_loss_list,
        "val_loss_list":val_loss_list,
        "epoch":epoch
    }

    torch.save(model, f"./saved_model/{name}_checkpoint.pt")


### Trying to load the saved model

In [42]:
loaded_model = torch.load(f"./saved_model/{model_name}.pt")

In [45]:
loaded_model(next(iter(train_dataloaders))[0])

tensor([[-3.5725, -3.0026, -3.7687,  ..., -2.6252, -2.2044, -2.0868],
        [-3.9351, -3.3094, -4.1523,  ..., -2.8935, -2.4316, -2.3009],
        [-4.1794, -3.5117, -4.4124,  ..., -3.0744, -2.5797, -2.4438],
        ...,
        [-4.1120, -3.4586, -4.3463,  ..., -3.0258, -2.5404, -2.4033],
        [-3.4633, -2.9185, -3.6615,  ..., -2.5491, -2.1438, -2.0248],
        [-3.4163, -2.8737, -3.6069,  ..., -2.5167, -2.1125, -1.9998]],
       grad_fn=<AddmmBackward0>)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
loaded_model = torch.load(f"./saved_model/{model_name}.pt")

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes